In [1]:
import openai
import os
from os import path
import json

Insert your API key

Can be created at https://platform.openai.com/account/api-keys

In [2]:
openai.api_key = os.environ["OPENAI_API_KEY"]
#openai.api_key = 'sk-...'

Helper functions

In [3]:
def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0, # this is the degree of randomness of the model's output, between 0 and 2
    )
    return response.choices[0].message["content"]

def get_completion_from_messages(messages, model="gpt-3.5-turbo", temperature=0):
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature, 
    )
    return response.choices[0].message["content"]

# The following function collects all previous messages when a new one is sent 
def collect_messages(_):
    prompt = inp.value_input
    inp.value = ''
    context.append({'role':'user', 'content':f"{prompt}"})
    response = get_completion_from_messages(context, temperature=0.2) 
    context.append({'role':'assistant', 'content':f"{response}"})
    panels.append(
        pn.Row('User:', pn.pane.Markdown(prompt, width=600)))
    panels.append(
        pn.Row('PlantBot:', pn.pane.Markdown(response, width=600, styles={'background-color': '#950100'})))
 
    return pn.Column(*panels)

The following code give the context to chatGPT in order to become PlantBot, the automated technical support service

In [4]:
import panel as pn  # GUI

initialContext = [ {'role':'system', 'content':"""
You are PlantBot, an automated technical support service for the company Tinyfarms. \
First here is information about the company Tinyfarms and their product \
The company sells a product called Growbot, which is essentially a small sustainable farm. \
These Growbots are sold to middle-school teacher for educational purposes. \
The Growbots kits include \
sensors which are camera, temperature sensor, humidity sensor, CO2 level sensor, and water level sensor \
full-spectrum LEDs \
pumps for water and for nutrients \
pot holders \
27 plant pods, 9 of oregano, 9 of basil and 9 of dill \
nutrient packs. \
The teachers and their students have access to a web application that includes controls, analytics, low-level programming and courses. \
The controls page allows users to activate the pumps and use the camera. \
The analytics page gives access to the sensors and some analytics. \
The programming is very low-level, friendly for middle schoolers. \
The courses promote knowledge about sustainability, programming, and technology. \
The different chapters of the course are \
1 Photosynthesis \
2 Nutrients in the soils \
3 Sustainable farming \
4 Basic programming \
5 Pump mechanics \
6 How do sensors work \
Your tasks as PlantBot are the following: \
You first greet the user by their name, introduce yourself, then ask which they are, middle-school student or teacher. \
Students can identify themselves by only saying their grade. \
Always keep in mind who you are talking to. \
Once you have identified your interlocutor, you will ask how you can be of service. \
You will keep a consistent tone and vocabulary adapted to the user you are facing. \
For technical support requests, you will ask questions to find the root of the problems. \
You will then offer one or two solutions. \
You will make sure that the problem is solved or that the user requests to talk to a Tinyfarm employee. \
If and only if they request to talk to an employee you will give them the following email address \
support@tinyfarms.example. \
Only teachers should be given access to that email address. \
For help or advices requests, you will reply as precisely as possible and then refer the user to a chapter of the course. \
Your technical support is only limited to Tinyfarms, their Growbots and the web application and the topics of the course. \
After giving your final answer, you will ask if the user needs further support and provide it if necesarry. \
Only when no support is needed will you politely close the conversation. \
"""} ]

The following function returns the context for each specific user.

New users will be introduced to the service while existing users will have their chat history saved.

In [5]:
def returnContext(user,initialContext):
    if path.exists(f'{user}.json'): # Checking if the user already has a conversation history
        f = open(f'{user}.json')
        context = json.load(f)
        context.append({'role':'system', 'content':f'Resume the previous conversation. First greet the user by their name, {user}, then offer your assistance'},)
    else:
        context = initialContext.copy()
        context[0]['content']+=f'The name of the user you are interacting with is {user}. Remember to greet them first'
    return context

Run the following code to interact with the chatbot

In [6]:
name = input('Enter your name:')

context = returnContext(name,initialContext)

pn.extension()
panels = []
inp = pn.widgets.TextInput(value="Hi", placeholder='Enter text here…')
button_conversation = pn.widgets.Button(name="Chat!")

interactive_conversation = pn.bind(collect_messages, button_conversation)

dashboard = pn.Column(
    inp,
    pn.Row(button_conversation),
    pn.panel(interactive_conversation, loading_indicator=True, height=300), scroll = True
    )

print('Current user: ',name)
dashboard

Current user:  John


BokehModel(combine_events=True, render_bundle={'docs_json': {'c3c4b5ca-6eaf-4473-828d-a9bdb79532f7': {'version…

This simple code saves the current conversation into a JSON file so that it could be used to resume the conversation at a later point

In [11]:
# Create new JSON file
conversation = context.copy()
with open(f'{name}.json', 'w') as f:
    json.dump(conversation, f, indent=2)
  
# Closing file
f.close()

Here we could ask chatGPT to do something with the messages, like building a report for Tinyfarms employees

In [14]:
messages =  context.copy()
messages.append(
{'role':'system', 'content':'Write a summary of the previous conversation between the user and you, the assistant.\
Be precise and use at most 150 words'},
)
response = get_completion_from_messages(messages, temperature=0)
for i in response.split('. '):
    print(i)

In this conversation, John, a middle-school student, contacted PlantBot, an automated technical support service for Tinyfarms, to report that the pump in his Growbot was not working
PlantBot provided several troubleshooting solutions, including checking the pump's connection to the power source and water/nutrient container, cleaning the pump and tubes, and checking the pump's activation on the web application
After trying these solutions, John reported that he could hear the pump turning on but no water was flowing
PlantBot then suggested cleaning the tubes and pump to remove any clogs or debris
After following these instructions, John reported that the pump was working again
Finally, John asked why the pump was necessary, and PlantBot explained that the pump helps to circulate water and nutrient solution throughout the plant pods, ensuring that all plants receive the necessary resources for healthy growth.
